# Migration WorkFlow Example

This is a more complex scenario that describes a workflow where there are two service engine groups that need to be merged into a single one. We will define a **TARGET** service engine group that will be used to receive the configuration extracted from the **SOURCE** service engine group. 

<img src="images/migration_scenario.png">

- [1.- Initial Login](#1--initial-login)
- [2.- Source Service Engine Data Collection](#step-1--collecting-source-service-engine-group-data)


- STEP 1.- Read Source SE Group information

- STEP 2.- Select interface to be migrated from source SE

- STEP 3.- Creating Cloned objects at target SE Group 
   - VRF_NEW
   - NETWORK_NEW
   - NETWORK_SERVICE_NEW
   - INTERFACE_CONFIG 
   - VSVIPs_NEW
   - POOLS_NEW
   - VIRTUAL_SERVICES_NEW (DISABLED STATE)

- STEP 4.- Proceed with Migration 
   - 1 Disabling interfaces at Source SE 
   - 2 Disabling VS at Source
   - 3 Enabling VS at TARGET SEG

- STEP 5.- Verification
          - Possible Roll-Back (optional)

- STEP 6.- Cleaning old configurations and renaming to original names
        - Delete migrated VSs
        - Delete migrated VSVIPs
        - Delete migrated Network Service
        - Delete migrated Network
        - Delete migrated VRF
        - Delete migrated Service Engines
        - Delete migrated Service Engine Group
        - Restoring original names 

## Initial Login

In [ ]:
from avi.sdk.avi_api import ApiSession
import datetime, time
from requests.packages import urllib3
urllib3.disable_warnings()
import json
import pandas as pd
from IPython.display import display, Image

# Import custom libraries
from libs.aux import *
from libs.extract import *
from libs.clone import *

# Import environment variables with controller information and credentials
from envs.controller_info import session_params as session_env

# Establish a first session with AVI Controller
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Update headers and api version imported from demo env file with controller version (ensure actual API Version is uses in subsequent requests
session_env['headers']['X-Avi-Version'] = api.remote_api_version['Version']
session_env['api_version'] = api.remote_api_version['Version']

# Create a new session with received AVI API Version
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Display Session ID to Verify AVI Controller Session Establishment
print('Successful connection to ' + session_env['name'] + '. Session ID:' + api.session_id)

### Select  SOURCE and TARGET Service Engine Group Names using Interactive Menu
Sometimes options might not be displayed. If so, stop the cell execution and restart. As last resort, create variables manually

```
source_segroup = "SEG-SOURCE-MAD-002"
target_segroup = "SEG-TARGET-MAD-001" 
```

In [ ]:
# Get SOURCE AND TARGET Service Engine Group Names from the list using menu
query = {
    "fields": "name"
}

resp = api.get("serviceenginegroup", params=query)
resp = json.loads(resp.text)["results"]

menu_options = []
i = 0
for item in resp:
    menu_options.insert(i, "Service Engine Group: "+item["name"])
    i = i +1

menu_title = "Please Select an option for SOURCE Service Engine Group to be migrated:"
selected_option = display_menu_from_list(menu_options, menu_title)
    
if selected_option:
    print("You selected: \033[1m"+selected_option+"\033[0m as SOURCE")
else:
    print("No valid option was selected.")

source_segroup = selected_option.split(": ")[1]

print()
menu_title = "Please Select an option for TARGET Service Engine Group:"
selected_option = display_menu_from_list(menu_options, menu_title)

if selected_option:
    print("You selected: \033[1m"+selected_option+"\033[0m as TARGET")
    if selected_option.split(": ")[1] == source_segroup:
        while selected_option.split(": ")[1] == source_segroup:
          print()
          print (" !!!!! SOURCE AND TARGET Service Engine Groups cannot be the same !!!!! Repeat Selection for TARGET...")
          menu_title = "Please Select an different option for TARGET Service Engine Group:"
          selected_option = display_menu_from_list(menu_options, menu_title)
          print(selected_option)
else:
    print("No valid option was selected.")

target_segroup = selected_option.split(": ")[1] 

In [ ]:
# Set SOURCE AND TARGET Service Engine Group Names from the list above
# Uncomennt and populate both variables if interactive menu does not work
#source_segroup = "SEG-SOURCE-MAD-002"
#target_segroup = "SEG-TARGET-MAD-001"

## STEP 1.- Collecting Source Service Engine Groups and associated Service Engines Information

The first step is to gather SE Group Information
- Service Engine Group Configuration
- Service Engine Configuration

In [ ]:
# Extracting source info
print("\033[1mExtracting SOURCE Service Engine Group Information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")
source_segroup_data = extract_segroup_data(session_env, source_segroup)
source_se_data = extract_se_data_from_segroup(session_env, source_segroup_data)

# Extracting target info
print("\033[1mExtracting TARGET Service Engine Group Information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")
target_segroup_data = extract_segroup_data(session_env, target_segroup)
target_se_data = extract_se_data_from_segroup(session_env, target_segroup_data)

## STEP 2.- Explore SOURCE collected information and select the source interface to be migrated

In [47]:
# Exploring Interface Acapters Information of SOURCE SE
source_if_names=[]
for index in range(len(source_se_data)):
    print("\033[1mShowing information of the SOURCE Service Engine "+str(index+1)+" \033[0m")
    print("\033[1m--------------------------------------------------\033[0m")
    se_data_vnics = source_se_data[index]["data_vnics"]
    se_df = pd.DataFrame(se_data_vnics)
    display(se_df)
    menu_options=[]
    for i in range(len(se_data_vnics)):
        if "vnic_networks" in se_data_vnics[i]:
            if_name = se_data_vnics[i]["if_name"]
            vrf_name = se_data_vnics[i]["vrf_ref"].split("#")[1]
            mac_address = se_data_vnics[i]["mac_address"]
            ip_addr = se_data_vnics[i]["vnic_networks"][0]["ip"]["ip_addr"]["addr"]
            type = se_data_vnics[i]["vnic_networks"][0]["ip"]["ip_addr"]["type"]
            mask = se_data_vnics[i]["vnic_networks"][0]["ip"]["mask"]
            mode = se_data_vnics[i]["vnic_networks"][0]["mode"]
            menu_option = "Interface "+ if_name+" at VRF "+vrf_name+" with mac "+ mac_address+" and IP"+type+" ADDRESS "+ip_addr+"/"+str(mask)
            menu_options.append(menu_option)
    menu_title = "Please Select a candidate interface for SOURCE Service Engine "+str(index+1)+" from above list to be migrated:"
    print()
    selected_option = display_menu_from_list(menu_options, menu_title)
    if selected_option:
        selected_if_name = selected_option.split(" ")[1]
        print("You selected: \033[1m"+selected_if_name+"\033[0m as SOURCE interface for Service Engine "+str(index+1))
        print()
        source_if_names.insert(index,selected_if_name)
    else:
        print("No valid option was selected.")


Showing information of the SOURCE Service Engine 1 
--------------------------------------------------


,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vrf_ref,vnic_networks
0,Unknown,True,False,eth3,False,eth3,00:0c:29:a7:83:62,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
1,Unknown,True,False,eth4,False,eth6,00:0c:29:a7:83:6c,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
2,Unknown,True,False,eth6,False,eth8,00:0c:29:a7:83:76,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
3,Unknown,True,False,eth8,False,eth1,00:0c:29:a7:83:80,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
4,Unknown,True,False,eth1,False,eth4,00:0c:29:a7:83:8a,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,"[{'ip': {'ip_addr': {'addr': '192.168.2.22', '..."
5,Unknown,True,False,eth2,False,eth7,00:0c:29:a7:83:94,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
6,Unknown,True,False,eth5,False,eth9,00:0c:29:a7:83:9e,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
7,Unknown,True,False,eth7,False,eth2,00:0c:29:a7:83:a8,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
8,Unknown,True,False,eth9,False,eth5,00:0c:29:a7:83:b2,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN



Please Select a candidate interface for SOURCE Service Engine 1 from above list to be migrated:
--------------------------------------------------
1. Interface eth1 at VRF vrf_source_mad_002 with mac 00:0c:29:a7:83:8a and IPV4 ADDRESS 192.168.2.22/24
You selected: eth1 as SOURCE for Service Engine 1

Showing information of the SOURCE Service Engine 2 
--------------------------------------------------


,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vrf_ref,vnic_networks
0,Unknown,True,False,eth3,False,eth3,00:0c:29:a7:61:16,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
1,Unknown,True,False,eth4,False,eth6,00:0c:29:a7:61:20,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
2,Unknown,True,False,eth6,False,eth8,00:0c:29:a7:61:2a,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
3,Unknown,True,False,eth8,False,eth1,00:0c:29:a7:61:34,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
4,Unknown,True,False,eth1,False,eth4,00:0c:29:a7:61:3e,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,"[{'ip': {'ip_addr': {'addr': '192.168.2.23', '..."
5,Unknown,True,False,eth2,False,eth7,00:0c:29:a7:61:48,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
6,Unknown,True,False,eth5,False,eth9,00:0c:29:a7:61:52,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
7,Unknown,True,False,eth7,False,eth2,00:0c:29:a7:61:5c,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
8,Unknown,True,False,eth9,False,eth5,00:0c:29:a7:61:66,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN



Please Select a candidate interface for SOURCE Service Engine 2 from above list to be migrated:
--------------------------------------------------
1. Interface eth1 at VRF vrf_source_mad_002 with mac 00:0c:29:a7:61:3e and IPV4 ADDRESS 192.168.2.23/24
You selected: eth1 as SOURCE for Service Engine 2



## STEP 3.- Extract information from selected interfaces

In [ ]:
# source interfaces should be selected using above interactive menu
# Uncomennt and populate both variables if interactive menu does not work using this sample format:
# Typically both names should be the same from a given SE HA pair
# source_if_names = ["eth1", "eth1"]

In [ ]:
interface_config = extract_interface_data(session_env, source_se_data, source_if_names)
output_config = interface_config
print("\033[1mExtracting SOURCE Virtual Services Information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")
source_vs_data = extract_vss_data(session_env, source_se_data, interface_config)

# Update variable with extracted info
output_config["virtualservices"] = source_vs_data

print("\033[1mExtracting SOURCE VSVIPs and Related Network Placement Information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")
network_vsvips_config = extract_network_vsvip_data(session_env, source_vs_data)
# Update variable with extracted info
output_config["network_config"]=network_vsvips_config["network_config"]
output_config["vsvips_config"]=network_vsvips_config["vsvips_config"]

In [ ]:
print (" Showing full extracted config...")
print(json.dumps(output_config, indent=3, sort_keys=False))

## STEP 4.- Create cloned objects from selected configuration

 - Clone VRF
 - Clone Placement Networks
 - Clone Network Services (if any)
 - 

In [ ]:
# 1 Clone existing VRF object
# New objects will take a name in the form <original_name>-NEW
# Returns a new dictionary with cloned object 
source_vrf = output_config["ip_routing_config"]["vrfcontext"]
cloned_vrf = clone_vrf (session_env, source_vrf)
print()
print("\033[1mCheck returned output for further verification:\033[0m")
print("\033[1m-----------------------------------------------\033[0m")
print(json.dumps(cloned_vrf, indent=3, sort_keys=True))

In [ ]:
# 2 Clone network object in the previously cloned VRF
# New objects will take a name in the form <original_name>-NEW
# Returns a new dictionary with cloned object 
target_vrf = cloned_vrf["name"]
source_networks = output_config["network_config"]
cloned_networks = []
for source_network in source_networks:
  cloned_network = clone_network (session_env, source_network, target_vrf)
  cloned_networks.append(cloned_network)
print()
print("\033[1mCheck returned output for further verification:\033[0m")
print("\033[1m-----------------------------------------------\033[0m")
print(json.dumps(cloned_networks, indent=3, sort_keys=True))

In [ ]:
# 3 Clone existing networkservice object in the previously cloned VRF in the target service engine group. 
# Returns a new dictionary with cloned object configuration 
target_vrf = cloned_vrf["name"]
if output_config["network_service_config"]:
    source_networkservice = output_config["network_service_config"]["name"]
    cloned_networkservice = clone_networkservice (session_env, source_networkservice, target_vrf, target_segroup )
    print()
    print("\033[1mCheck returned output for further verification:\033[0m")
    print("\033[1m-----------------------------------------------\033[0m")
    print(json.dumps(cloned_networkservice, indent=3, sort_keys=True))
else:
    print("\033[1mNo existing NetworkService to clone :\033[0m")
    print("\033[1m-------------------------------------\033[0m")

In [ ]:
# 4 Clone existing VSVIPs objects extracted from a source VRF into the previously cloned VRF into the target service engine group. 
# Returns a" new dictionary with cloned object configuration
source_vrf=output_config["ip_routing_config"]["vrfcontext"]
target_vrf=cloned_vrf["name"]
cloned_vsvips=[]
for vsvip in output_config["vsvips_config"]:
    target_network = vsvip["placement_network"]+"-NEW"
    vsvip_name = vsvip["name"]
    cloned_vsvip = clone_vsvips(session_env, vsvip_name, target_vrf, target_network)
    cloned_vsvips.append(cloned_vsvip)
if (cloned_vsvips):
    print("\033[1mNew cloned objects summary for network"+target_network+"\033[0m")
    print("\033[1m------------------------------------------------------\033[0m")
    for item in cloned_vsvips:
      print ("VSVIP named \033[1m"+item["name"]+"\033[0m with IP Address \033[1m"+item["vip"][0]["ip_address"]["addr"]+"\033[0m at target VRF \033[1m"+ target_vrf+"\033[0m")
print()
print("\033[1mCheck full returned output for further verification:\033[0m")
print("\033[1m----------------------------------------------------\033[0m")
print(json.dumps(cloned_vsvips, indent=3, sort_keys=True))

In [ ]:
output_config["virtualservices"]

In [ ]:
# 5 Clone Pools related to an VS existing in a given source VRF and duplicate into a target_vrf 
# Returns a new dictionary with cloned object configuration
source_vrf=output_config["ip_routing_config"]["vrfcontext"]
target_vrf=cloned_vrf["name"]
cloned_pools = []
for vs in output_config["virtualservices"]:
    vs_name = vs["name"]
    cloned_pool = clone_pools (session_env, vs, target_vrf)
    cloned_pools.append(cloned_pool)
if (cloned_pools):
    print("\033[1mNew cloned objects summary\033[0m")
    print("\033[1m-------------------------\033[0m")
    for pool in cloned_pools:
        print ("Pool named \033[1m"+pool["name"]+"\033[0m with following servers:")
        for server in pool["servers"]:
           print (" - Server IP "+server["ip"]["addr"])
print()
print("\033[1mCheck full returned output for further verification:\033[0m")
print("\033[1m----------------------------------------------------\033[0m")
print(json.dumps(cloned_pools, indent=3, sort_keys=True))

In [ ]:
# 6 Clone VirtualServices from a given source_vrf and segroup
# Returns a new dictionary with cloned object configuration
source_vrf=output_config["ip_routing_config"]["vrfcontext"]
#target_vrf=cloned_vrf["name"]
#target_vrf= "vrf_source_mad_002-NEW"
#target_segroup = "SEG-TARGET-MAD-001"
cloned_vss=[]
for source_vs in output_config["virtualservices"]:
    source_vs_name = source_vs["name"]
    cloned_vs = clone_virtualservices (session_env, source_vs, target_vrf, target_segroup)
    cloned_vss.append(cloned_vs)
if (cloned_vss):
    print("\033[1mNew cloned VirtualServices summary\033[0m")
    print("\033[1m-------------------------\033[0m")
    for item in cloned_vss:
        print ("Virtual Service name \033[1m"+item["name"]+"\033[0m with following config:")
        print (" - VRF Context name "+item["vrf_context_ref"].split("#")[1])
        print (" - VSVIP name "+item["vsvip_ref"].split("#")[1])
        print (" - Pool name "+item["pool_ref"].split("#")[1])
print()
print("\033[1mCheck full returned output for further verification:\033[0m")
print("\033[1m----------------------------------------------------\033[0m")
print(json.dumps(cloned_vss, indent=3, sort_keys=True))

### TARGET SERVICE ENGINE GROUP

In [43]:
# Exploring Interface Acapters Information of TARGET SEs
target_if_names=[]
for index in range(len(target_se_data)):
    print("\033[1mShowing information of the TARGET Service Engine "+str(index+1)+" \033[0m")
    print("\033[1m--------------------------------------------------\033[0m")
    se_data_vnics = target_se_data[index]["data_vnics"]
    se_df = pd.DataFrame(se_data_vnics)
    display(se_df)
    menu_options=[]
    for i in range(len(se_data_vnics)):
        if "vnic_networks" not in se_data_vnics[i]:
            if_name = se_data_vnics[i]["if_name"]
            vrf_name = se_data_vnics[i]["vrf_ref"].split("#")[1]
            #mac_address = se_data_vnics[i]["mac_address"]
            #ip_addr = se_data_vnics[i]["vnic_networks"][0]["ip"]["ip_addr"]["addr"]
            #type = se_data_vnics[i]["vnic_networks"][0]["ip"]["ip_addr"]["type"]
            #mask = se_data_vnics[i]["vnic_networks"][0]["ip"]["mask"]
            #mode = se_data_vnics[i]["vnic_networks"][0]["mode"]
            menu_option = "Interface "+ if_name+" at VRF "+vrf_name+" has no current IP information and it appears available "
            menu_options.append(menu_option)
    menu_title = "Please Select a candidate interface for TARGET Service Engine "+str(index+1)+" from above list receive new config:"
    print()
    selected_option = display_menu_from_list(menu_options, menu_title)
    print(selected_option)
    if selected_option:
        selected_if_name = selected_option.split(" ")[1]
        print("You selected: \033[1m"+selected_if_name+"\033[0m as TARGET for Service Engine "+str(index+1))
        print()
        target_if_names.insert(index,selected_if_name)
    else:
        print("No valid option was selected.")

Showing information of the TARGET Service Engine 1 
--------------------------------------------------


,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vrf_ref,vnic_networks
0,Unknown,True,False,eth9,False,eth5,00:0c:29:c4:a7:09,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
1,Unknown,True,False,eth3,False,eth3,00:0c:29:c4:a7:b9,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
2,Unknown,True,False,eth4,False,eth6,00:0c:29:c4:a7:c3,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
3,Unknown,True,False,eth6,False,eth8,00:0c:29:c4:a7:cd,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
4,Unknown,True,False,eth8,False,eth1,00:0c:29:c4:a7:d7,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
5,Unknown,True,False,eth1,False,eth4,00:0c:29:c4:a7:e1,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,"[{'ip': {'ip_addr': {'addr': '192.168.1.40', '..."
6,Unknown,True,False,eth2,False,eth7,00:0c:29:c4:a7:eb,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
7,Unknown,True,False,eth5,False,eth9,00:0c:29:c4:a7:f5,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
8,Unknown,True,False,eth7,False,eth2,00:0c:29:c4:a7:ff,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN



Please Select a candidate interface for TARGET Service Engine 1 from above list receive new config:
--------------------------------------------------
1. Interface eth9 at VRF global has no current IP information and it appears available 
2. Interface eth3 at VRF global has no current IP information and it appears available 
3. Interface eth4 at VRF global has no current IP information and it appears available 
4. Interface eth6 at VRF global has no current IP information and it appears available 
5. Interface eth8 at VRF global has no current IP information and it appears available 
6. Interface eth2 at VRF global has no current IP information and it appears available 
7. Interface eth5 at VRF global has no current IP information and it appears available 
8. Interface eth7 at VRF global has no current IP information and it appears available 
Interface eth4 at VRF global has no current IP information and it appears available 
You selected: eth4 as TARGET for Service Engine 1

Showing 

,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vrf_ref,vnic_networks
0,Unknown,True,False,eth6,False,eth8,00:0c:29:ba:47:09,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
1,Unknown,True,False,eth8,False,eth1,00:0c:29:ba:47:13,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
2,Unknown,True,False,eth1,False,eth4,00:0c:29:ba:47:1d,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,"[{'ip': {'ip_addr': {'addr': '192.168.1.41', '..."
3,Unknown,True,False,eth2,False,eth7,00:0c:29:ba:47:27,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
4,Unknown,True,False,eth5,False,eth9,00:0c:29:ba:47:31,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
5,Unknown,True,False,eth7,False,eth2,00:0c:29:ba:47:3b,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
6,Unknown,True,False,eth9,False,eth5,00:0c:29:ba:47:45,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
7,Unknown,True,False,eth3,False,eth3,00:0c:29:ba:47:f5,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
8,Unknown,True,False,eth4,False,eth6,00:0c:29:ba:47:ff,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN



Please Select a candidate interface for TARGET Service Engine 2 from above list receive new config:
--------------------------------------------------
1. Interface eth6 at VRF global has no current IP information and it appears available 
2. Interface eth8 at VRF global has no current IP information and it appears available 
3. Interface eth2 at VRF global has no current IP information and it appears available 
4. Interface eth5 at VRF global has no current IP information and it appears available 
5. Interface eth7 at VRF global has no current IP information and it appears available 
6. Interface eth9 at VRF global has no current IP information and it appears available 
7. Interface eth3 at VRF global has no current IP information and it appears available 
8. Interface eth4 at VRF global has no current IP information and it appears available 
Interface eth2 at VRF global has no current IP information and it appears available 
You selected: eth2 as TARGET for Service Engine 2



### Select one FREE interface at Target Service Engines that interface that will be used to receive gathered ip address configuration from source interface at source Service Engines


In [ ]:
# manual selection
# target_if_names = ["eth2", "eth2"]

In [ ]:
# Assuming IP Address will be REUSED!!!
#  1.- Apply SE Interface configuration at new VRF in target Service Engines (keep disabled)
#  2.- Disable migrated interfaces at SOURCE SEs
#  3.- Enable new configured interfaces at TARGET SEs
#  4.- Verify new VS are working

## 2.1  Migrate Service Engine Interfaces

In [ ]:
# Display gathered information prior to execute next cell


print ("The following interfaces has been selected for migration")
print()
for i in range(len(output_config)):
    print("For the SOURCE Service Engine "+str(i+1)+" with name \033[1m" + source_se_data[i]["name"]+"\033[0m")
    print("The interface \033[1m" + source_if_name + "\033[0m has been selected with following configuration")
    print("   IP Address \033[1m" + output_config[i]["ip_addr"] +"\033[0m")
    print("   Mask Length \033[1m" + str(output_config[i]["mask"]) +"\033[0m")
    print()
print("---------------------")
for i in range(len(target_se_data)):
    print("For the TARGET Service Engine "+str(i+1)+" with name \033[1m" + target_se_data[i]["name"]+"\033[0m")
    print("The interface \033[1m" + target_if_name + "\033[0m will be configured with above information.")
    print("   IP Address \033[1m" + output_config[i]["ip_addr"] +"\033[0m")
    print("   Mask Length \033[1m" + str(output_config[i]["mask"]) +"\033[0m")
    print()
print("---------------------")
print("WARNING!!! Before proceeding migrated interfaces in the source SEs will be disabled")

In [50]:
se_pairs=[]
for i in range(len(source_se_data)):
   se_pair = {
      "source_se_uuid": source_se_data[i]["uuid"],
      "source_if_name": source_if_names[i],
      "target_se_uuid": target_se_data[i]["uuid"],
      "target_if_name": target_if_names[i]
      }
   se_pairs.insert(i, se_pair)

In [51]:
se_pairs

[{'source_se_uuid': 'se-564d63c6-3882-ca52-7be5-b04c9ba78358',
  'source_if_name': 'eth1',
  'target_se_uuid': 'se-564dcb90-ff93-dc99-bf4a-62907bc4a7af',
  'target_if_name': 'eth4'},
 {'source_se_uuid': 'se-564d0f61-3a27-ff26-8af7-306296a7610c',
  'source_if_name': 'eth1',
  'target_se_uuid': 'se-564d5830-e7a9-edc0-194c-c76236ba47eb',
  'target_if_name': 'eth2'}]

In [ ]:
# Create dictionary list containing pair of migrating-target SEs  
se_pairs=[]
for i in range(len(source_se_data)):
   se_pair = {
      "source_se_uuid": source_se_data[i]["uuid"],
      "source_if_name": source_if_names[i],
      "target_se_uuid": target_se_data[i]["uuid"],
      "target_if_name": target_if_names[i]
      }
   se_pairs.insert(i, se_pair)
   
# Loop to migrate 2 se Pairs (migrating to target) 
for i in range(len(se_pairs)):
    source_se_uuid = se_pairs[i]["source_se_uuid"]
    target_se_uuid = se_pairs[i]["target_se_uuid"]
    print("Migrating pair "+str(i+1))
    print("-------------------------")
    print("Migrating (source) Service Engine UUID \033[1m"+source_se_uuid +"\033[0m at SE Group \033[1m"+source_segroup+"\033[0m") 
    print("Receiving (target) Service Engine UUID \033[1m"+target_se_uuid+"\033[0m at SE Group \033[1m"+target_segroup+"\033[0m")
    source_se_uuid = se_pairs[i]["source_se_uuid"]
    target_se_uuid = se_pairs[i]["target_se_uuid"]

    # Disabling interface to migrate at target SE
    ## Create function to set interface DISABLED for a given SE UUID
    # set_interface_disabled[]

    # Extract the source/target SE data with matching uuid
    source_se_json_data = [ se for se in source_se_data if se.get("uuid") == source_se_uuid]
    target_se_json_data = [ se for se in target_se_data if se.get("uuid") == target_se_uuid]


    # Extract source_interface vnic_networks object
    json_data_source_adapter = [ data_vnic for data_vnic in source_se_json_data[0]["data_vnics"] if data_vnic.get("if_name") == source_if_name ]
    json_data_source_adapter_vnic_networks = [json_data_source_adapter[0]["vnic_networks"][0]]

    # Modifying existing value for selected interface at source SE (i.e Enabled = False for corresponding interface to shutdown )
    # vnic_network list must be removed to avoid SE interface overlapping
    if_name_key = "if_name"
    if_name_value = source_if_name

    # Loop through each dictionary in the list
    for adapter in source_se_json_data[0]["data_vnics"]:
        # Check if the key_to_check matches the value_to_match
        if adapter.get(if_name_key) == if_name_value:
            # Update the value of key_to_modify
            adapter["enabled"] = False
            adapter["vnic_networks"]=[]
    
    # Remove _last_modified key
    source_se_json_data[0].pop("_last_modified", None)

    # Disable migrating interfaces to avoid IP address overlapping
    body = source_se_json_data[0]
    print ("Applying changes to source SE "+str(i+1)+" with uuid "+ source_se_uuid)
    print (" - Changing interface "+source_if_name+" to disabled state")
    print (" - Removing IP Address Configuration")

    url_path = "serviceengine/"+source_se_uuid
    resp = api.put (url_path, data=json.dumps(body))

    if resp.status_code in range(200, 299):
      print(resp)
      print('- Object '+url_path+' named '+body['name']+ " modified", resp.reason)#, resp.text)
      print()
    else:
      print('Error in modifying '+url_path+' :%s' % resp.text)

    # PUT to configure migrating IP Addresses in target SE
    # Extract IP Address from migrating SE
    # Importing extracted information into target body
    if_name_key = "if_name"
    if_name_value = target_if_name

    # Loop through each dictionary in the list
    for item in target_se_json_data[0]["data_vnics"]:
      # Check if the interface name correspondes to given value
      if item.get(if_name_key) == if_name_value:
        # Update the value of vnic_networks from source SE
        item["vnic_networks"] = json_data_source_adapter_vnic_networks
    
    # Remove _last_modified key
    target_se_json_data[0].pop("_last_modified", None)

    body = target_se_json_data[0]
    print ("Applying changes to target SE "+str(i+1)+" with uuid "+ target_se_uuid)
    print (" - Applying IP Address Configuration")
    print(target_se_json_data[0])

    url_path = "serviceengine/"+target_se_uuid
    resp = api.put (url_path, data=json.dumps(body))

    if resp.status_code in range(200, 299):
      print(resp)
      print('- Object '+url_path+' named '+body['name']+ " modified", resp.reason)#, resp.text)
      print()
    else:
      print('Error in modifying '+url_path+' :%s' % resp.text)

## 2.1  Migrate Virtual Services

In [ ]:
# Increase the number of VS per SE to acommodate imported VS
target_max_vs_per_se = target_segroup_data["max_vs_per_se"]
count_of_source_imported_vs = len(source_vs_data)
count_of_current_target_vs = len(target_vs_data)
if ((count_of_source_imported_vs + count_of_current_target_vs) >= (target_max_vs_per_se - 5)):
    target_segroup_data["max_vs_per_se"] = count_of_source_imported_vs + count_of_current_target_vs + 5
    print ("Increase the number of VS per SE to "+str(target_segroup_data["max_vs_per_se"])+" to acommodate imported VSs ")
    body = target_segroup_data
    # Remove _last_modified key to avoid concurrent update error
    body.pop("_last_modified", None)

    url_path = "serviceenginegroup/"+target_segroup_data["uuid"]
    resp = api.put (url_path, data=json.dumps(body))

    if resp.status_code in range(200, 299):
       print(resp)
       print('- Object '+url_path+' named '+body['name']+ " modified", resp.reason)#, resp.text)
       print()
    else:
       print('Error in modifying '+url_path+' :%s' % resp.text)

In [ ]:
# Obtain the target se_group_ref (corresponds to key url of the serviceenginegroup object ) 
target_se_group_ref = target_segroup_data["url"]

for item in source_vs_data:
    item["se_group_ref"]=target_se_group_ref
    print ("Service engine group reference has been changed for Virtual Service "+ item["name"])
    print ("Applying new configuration")

    # Remove _last_modified key to avoide concurrent update error
    item.pop("_last_modified", None)

    url_path = "virtualservice/"+item["uuid"]
    resp = api.put (url_path, data=json.dumps(item))

    if resp.status_code in range(200, 299):
       print(resp)
       print('- Object '+url_path+' named '+body['name']+ " modified", resp.reason)#, resp.text)
       print()
    else:
       print('Error in modifying '+url_path+' :%s' % resp.text)